# SQL查询语言

In [10]:
import pandas as pd
from datetime import datetime

In [11]:
var_list = ['event_id','user_id','gmt_occur','client_ip','network','device_sign','info_1','info_2','ip_prov','ip_city',
            'mobile_oper_platform','version','card_cert_no','cert_prov','cert_city','card_bin_prov','card_bin_city',
            'card_mobile_prov','card_mobile_city','card_cert_prov','card_cert_city','is_one_people','operation_channel',
            'pay_scene','amt','is_peer_pay','opposing_id','income_card_no','income_card_cert_no','income_card_mobile',
            'income_card_bank_code','province','city','dt','is_fraud']

In [12]:
var_ana_list = ['user_id','client_ip','network','device_sign','info_1','info_2','ip_prov','ip_city',
            'mobile_oper_platform','version','card_cert_no','cert_prov','cert_city','card_bin_prov','card_bin_city',
            'card_mobile_prov','card_mobile_city','card_cert_prov','card_cert_city','is_one_people','operation_channel',
            'pay_scene','amt','is_peer_pay','opposing_id','income_card_no','income_card_cert_no','income_card_mobile',
            'income_card_bank_code','province','city']

## 滑窗SQL

### group_var
- user_id
- opposing_id
- client_ip
- device_sign
- info_1
- info_2

### 总体数量滑窗

In [13]:
def window_sql(var,date,i,group_var,date_gap=3,user='kevin'):
    """
    基于kevin_new_id_all_win_features表统计特征 滑窗交易次数
    """
    if i == 0:
        print("CREATE TABLE IF NOT EXISTS {4}_winFea_{5}_all_count_{3}d AS".format(var,date,i,date_gap,user,group_var))
    else:
        print("INSERT INTO TABLE {4}_winFea_{5}_all_count_{3}d ".format(var,date,i,date_gap,user,group_var))    
    print("""SELECT v.event_id,s.win_{5}_to_count_{3}_num
    FROM(
        SELECT 
            event_id,{5}
        FROM kevin_new_id_all_win_features    
        WHERE gmt_date=to_date('{1}','yyyy-mm-dd')
    ) v 
    LEFT OUTER JOIN (
        SELECT {5}, count(*) AS win_{5}_to_count_{3}_num
        FROM  kevin_new_id_all_win_features
        WHERE  gmt_date<to_date('{1}','yyyy-mm-dd') and gmt_date>=dateadd(to_date('{1}','yyyy-mm-dd'), -{3}, 'dd')
                and  {5} is not NULL
        GROUP BY {5}
    )s
    ON v.{5} = s.{5};
    """.format(var,date,i,date_gap,user,group_var))

### DISTINCT COUNT 统计

In [14]:
def window_distinct_count_sql(var,date,i,group_var,user_id,opposing_id,client_ip,device_sign,info_1,info_2,date_gap=3,user='kevin'):
    """
    基于kevin_new_id_all_win_features表统计特征
    group_var:需分析的变量
    user_id,opposing_id,client_ip,device_sign,info_1,info_2 为分析变量列表 统一传入 params用于判断是否注释
    """
    if i == 0:
        print("CREATE TABLE IF NOT EXISTS {4}_winFea_{5}_distinct_count_{3}d AS".format(var,date,i,date_gap,user,group_var))
    else:
        print("INSERT INTO TABLE {4}_winFea_{5}_distinct_count_{3}d ".format(var,date,i,date_gap,user,group_var))        
    print("""SELECT v.event_id,
        {6} s.win_{5}_to_user_id_{3}_num,
        {7} s.win_{5}_to_client_ip_{3}_num,
            s.win_{5}_to_network_{3}_num,
        {8} s.win_{5}_to_device_sign_{3}_num,
        {9} s.win_{5}_to_info_1_{3}_num,
       {10} s.win_{5}_to_info_2_{3}_num,
            s.win_{5}_to_ip_prov_{3}_num,
            s.win_{5}_to_ip_city_{3}_num,
            s.win_{5}_to_mobile_oper_platform_{3}_num,
            s.win_{5}_to_version_{3}_num,
            s.win_{5}_to_card_cert_no_{3}_num,
            s.win_{5}_to_cert_prov_{3}_num,
            s.win_{5}_to_cert_city_{3}_num,
            s.win_{5}_to_card_bin_prov_{3}_num,
            s.win_{5}_to_card_bin_city_{3}_num,
            s.win_{5}_to_card_mobile_prov_{3}_num,
            s.win_{5}_to_card_mobile_city_{3}_num,
            s.win_{5}_to_card_cert_prov_{3}_num,
            s.win_{5}_to_card_cert_city_{3}_num,
            s.win_{5}_to_is_one_people_{3}_num,
            s.win_{5}_to_operation_channel_{3}_num,
            s.win_{5}_to_pay_scene_{3}_num,
            s.win_{5}_to_amt_{3}_num,
            s.win_{5}_to_is_peer_pay_{3}_num,
       {11} s.win_{5}_to_opposing_id_{3}_num,
            s.win_{5}_to_income_card_no_{3}_num,
            s.win_{5}_to_income_card_cert_no_{3}_num,
            s.win_{5}_to_income_card_mobile_{3}_num,
            s.win_{5}_to_income_card_bank_code_{3}_num,
            s.win_{5}_to_province_{3}_num,
            s.win_{5}_to_city_{3}_num 
    FROM(
        SELECT 
            event_id,{5}
        FROM kevin_new_id_all_win_features    
        WHERE gmt_date=to_date('{1}','yyyy-mm-dd')
    ) v 
    LEFT OUTER JOIN (
        SELECT 
            {5},
        {6} count(DISTINCT user_id) AS win_{5}_to_user_id_{3}_num,    -- group_var = user_id 时注释
        {7} count(DISTINCT client_ip) AS win_{5}_to_client_ip_{3}_num,
            count(DISTINCT network) AS win_{5}_to_network_{3}_num,
        {8} count(DISTINCT device_sign) AS win_{5}_to_device_sign_{3}_num,
        {9} count(DISTINCT info_1) AS win_{5}_to_info_1_{3}_num,
       {10} count(DISTINCT info_2) AS win_{5}_to_info_2_{3}_num,
            count(DISTINCT ip_prov) AS win_{5}_to_ip_prov_{3}_num,
            count(DISTINCT ip_city) AS win_{5}_to_ip_city_{3}_num,
            count(DISTINCT mobile_oper_platform) AS win_{5}_to_mobile_oper_platform_{3}_num,
            count(DISTINCT version) AS win_{5}_to_version_{3}_num,
            count(DISTINCT card_cert_no) AS win_{5}_to_card_cert_no_{3}_num,
            count(DISTINCT cert_prov) AS win_{5}_to_cert_prov_{3}_num,
            count(DISTINCT cert_city) AS win_{5}_to_cert_city_{3}_num,
            count(DISTINCT card_bin_prov) AS win_{5}_to_card_bin_prov_{3}_num,
            count(DISTINCT card_bin_city) AS win_{5}_to_card_bin_city_{3}_num,
            count(DISTINCT card_mobile_prov) AS win_{5}_to_card_mobile_prov_{3}_num,
            count(DISTINCT card_mobile_city) AS win_{5}_to_card_mobile_city_{3}_num,
            count(DISTINCT card_cert_prov) AS win_{5}_to_card_cert_prov_{3}_num,
            count(DISTINCT card_cert_city) AS win_{5}_to_card_cert_city_{3}_num,
            count(DISTINCT is_one_people) AS win_{5}_to_is_one_people_{3}_num,
            count(DISTINCT operation_channel) AS win_{5}_to_operation_channel_{3}_num,
            count(DISTINCT pay_scene) AS win_{5}_to_pay_scene_{3}_num,
            count(DISTINCT amt) AS win_{5}_to_amt_{3}_num,
            count(DISTINCT is_peer_pay) AS win_{5}_to_is_peer_pay_{3}_num,
       {11} count(DISTINCT opposing_id) AS win_{5}_to_opposing_id_{3}_num,
            count(DISTINCT income_card_no) AS win_{5}_to_income_card_no_{3}_num,
            count(DISTINCT income_card_cert_no) AS win_{5}_to_income_card_cert_no_{3}_num,
            count(DISTINCT income_card_mobile) AS win_{5}_to_income_card_mobile_{3}_num,
            count(DISTINCT income_card_bank_code) AS win_{5}_to_income_card_bank_code_{3}_num,
            count(DISTINCT province) AS win_{5}_to_province_{3}_num,
            count(DISTINCT city) AS win_{5}_to_city_{3}_num
        FROM  kevin_new_id_all_win_features
        WHERE  gmt_date<to_date('{1}','yyyy-mm-dd') and gmt_date>=dateadd(to_date('{1}','yyyy-mm-dd'), -{3}, 'dd')
                and  {5} is not NULL
        GROUP BY {5}
    )s
    ON v.{5} = s.{5};
    """.format(var,date,i,date_gap,user,group_var,user_id,client_ip,device_sign,info_1,info_2,opposing_id,))


### 滑窗金额合计

In [15]:
def window_sum_amt_sql(var,date,i,date_gap=3,user='kevin'):
    """
    基于kevin_new_id_all_win_features表统计特征 滑窗交易金额和
    """
    if i == 0:
        print("CREATE TABLE IF NOT EXISTS {4}_winFea_{5}_sum_amt_{3}d AS".format(var,date,i,date_gap,user,group_var))
    else:
        print("INSERT INTO TABLE {4}_winFea_{5}_sum_amt_{3}d ".format(var,date,i,date_gap,user,group_var))   
    print("""SELECT v.event_id,
    (case when s.win_sum_amt_{3}_num is NULL then 0 else s.win_sum_amt_{3}_num end) as win_sum_amt_{3}_num
    FROM(
        SELECT 
            event_id,{5}
        FROM kevin_new_id_all_win_features    
        WHERE gmt_date=to_date('{1}','yyyy-mm-dd')
    ) v 
    LEFT OUTER JOIN (
        SELECT {5}, sum(amt) AS win_sum_amt_{3}_num
        FROM  kevin_new_id_all_win_features
        WHERE  gmt_date<to_date('{1}','yyyy-mm-dd') and gmt_date>=dateadd(to_date('{1}','yyyy-mm-dd'), -{3}, 'dd')
                and  {5} is not NULL
        GROUP BY {5}
    )s
    ON v.{5} = s.{5};
    """.format(var,date,i,date_gap,user,group_var))

In [16]:
def window_sum_amt_count_sql(var,date,i,group_var,date_gap=3,user='kevin'):
    """
    基于kevin_new_id_all_win_features表统计特征
    只统计过去滑窗的gmt_occur出现次数
    """
    if i == 0:
        print("CREATE TABLE IF NOT EXISTS {4}_winFea_{5}_distinct_gmt_count_{3}d AS".format(var,date,i,date_gap,user,group_var))
    else:
        print("INSERT INTO TABLE {4}_winFea_{5}_distinct_gmt_count_{3}d ".format(var,date,i,date_gap,user,group_var))       
    print("""SELECT v.event_id,
            s.win_gmt_occur_{3}_num,
            s.win_gmt_date_{3}_num
    FROM(
        SELECT 
            event_id,{5}
        FROM kevin_new_id_all_win_features    
        WHERE gmt_date=to_date('{1}','yyyy-mm-dd')
    ) v 
    LEFT OUTER JOIN (
        SELECT 
            {5},
            count(DISTINCT gmt_datetime) AS win_gmt_occur_{3}_num,
            count(DISTINCT gmt_date) AS win_gmt_date_{3}_num
        FROM  kevin_new_id_all_win_features
        WHERE  gmt_date<to_date('{1}','yyyy-mm-dd') and gmt_date>=dateadd(to_date('{1}','yyyy-mm-dd'), -{3}, 'dd')
                and  {5} is not NULL
        GROUP BY {5}
    )s
    ON v.{5} = s.{5};
    """.format(var,date,i,date_gap,user,group_var))


### 历史交易次数

In [17]:
def window_his_sql(var,date,i,group_var,date_gap=3,user='kevin'):
    """
    基于kevin_new_id_all_win_features表统计特征 之前历史交易次数
    """
    if i == 0:
        print("CREATE TABLE IF NOT EXISTS {4}_histFea_{5}_all_count AS".format(var,date,i,date_gap,user,group_var))
    else:
        print("INSERT INTO TABLE {4}_histFea_{5}_all_count".format(var,date,i,date_gap,user,group_var))    
    print("""SELECT v.event_id,s.hist_{5}_count_num
    FROM(
        SELECT 
            event_id,{5}
        FROM kevin_new_id_all_win_features    
        WHERE gmt_date=to_date('{1}','yyyy-mm-dd')
    ) v 
    LEFT OUTER JOIN (
        SELECT {5}, count(*) AS hist_{5}_count_num
        FROM  kevin_new_id_all_win_features
        WHERE  gmt_date<to_date('{1}','yyyy-mm-dd') and  source in ('train','test')
                and  {5} is not NULL
        GROUP BY {5}
    )s
    ON v.{5} = s.{5};
    """.format(var,date,i,date_gap,user,group_var))

In [18]:
def history_sql(var,var_ana_list,group_var,user='kevin'):
    """
    基于kevin_new_id_all_win_features表统计特征 滑窗交易次数
    """ 

    print("""
    CREATE TABLE IF NOT EXISTS  {2}_{1}_date_hist_count as        
    SELECT
        s.event_id,s.{1},""".format(var,group_var,user))

    for var in var_ana_list:
        if var not in group_var:
            print("""
                {0}.{0}, 
                {0}.rn-1  as hist_count_{0}_num,         
            """.format(var,group_var))
        
    print("""
    s.gmt_datetime
    FROM
    (SELECT *   
     FROM  kevin_new_id_all_win_features
      WHERE  source in ('train','test') ) s
    """.format(var,group_var))
        
    for var in var_ana_list:
        if var not in group_var:
            print("""
              LEFT OUTER JOIN 
            (
              SELECT 
                event_id, {1},{0}, gmt_datetime,
                rank() over(partition by {1},{0} order by gmt_datetime ) rn 
              from kevin_new_id_all_win_features
              WHERE  source in ('train','test')   and  {1} is not NULL
            ) {0}
            on s.event_id = {0}.event_id
            """.format(var,group_var))


In [19]:
def history_fenmu_sql(group_var,user='kevin'):

    # -- 统计时间差  以及过去交易情况——做分母
    print("""CREATE TABLE IF NOT EXISTS {1}_{0}_date_gap_count AS
      SELECT t.event_id 
           ,t.{0} 
           ,t.gmt_datetime 
           ,t.gmt_datetime_last 
           ,DATEDIFF(gmt_datetime ,gmt_datetime_last ,'hh') AS gmt_time_gap 
           ,(case when (t.rn-1) = 0 then 0.001 else (t.rn-1) end)             AS hist_{0}_count_num2 
       FROM (SELECT event_id 
                   ,{0} 
                   ,gmt_datetime 
                   ,LAG(gmt_datetime ,1) OVER ( PARTITION BY {0} ORDER BY gmt_datetime ) gmt_datetime_last 
                   ,RANK() OVER ( PARTITION BY {0} ORDER BY gmt_datetime ) rn                              -- ,
             --     row_number() over(partition by {0} order by gmt_datetime ) rn1
               FROM kevin_id_all_win_features 
              WHERE source IN ( 'train' , 'test' ) 
            )t 
    ;""".format(group_var,user)
    )

In [20]:
def history_prec_sql(group_var,df_fenzi,df_fenmu,user_id,opposing_id,client_ip,device_sign,info_1,info_2,user='kevin'):
    print("""
            CREATE TABLE IF NOT EXISTS {9}_prec_{0}_all_data as 
            SELECT s.event_id 
                  ,s.{0} 
                  ,s.gmt_datetime_last 
                  ,s.hist_{0}_count_num2
              {3} ,t.hist_count_user_id_num / s.hist_{0}_count_num2              AS hist_prec_{0}_user_id
              {4} ,t.hist_count_client_ip_num / s.hist_{0}_count_num2             AS hist_prec_{0}_client_ip 
                  ,t.hist_count_network_num / s.hist_{0}_count_num2               AS hist_prec_{0}_network 
              {5} ,t.hist_count_device_sign_num / s.hist_{0}_count_num2           AS hist_prec_{0}_device_sign 
              {6} ,t.hist_count_info_1_num / s.hist_{0}_count_num2                AS hist_prec_{0}_info_1 
              {7} ,t.hist_count_info_2_num / s.hist_{0}_count_num2                AS hist_prec_{0}_info_2 
                  ,t.hist_count_ip_prov_num / s.hist_{0}_count_num2               AS hist_prec_{0}_ip_prov 
                  ,t.hist_count_ip_city_num / s.hist_{0}_count_num2               AS hist_prec_{0}_ip_city 
                  ,t.hist_count_mobile_oper_platform_num / s.hist_{0}_count_num2  AS hist_prec_{0}_mobile_oper_platform 
                  ,t.hist_count_version_num / s.hist_{0}_count_num2               AS hist_prec_{0}_version 
                  ,t.hist_count_card_cert_no_num / s.hist_{0}_count_num2          AS hist_prec_{0}_card_cert_no 
                  ,t.hist_count_cert_prov_num / s.hist_{0}_count_num2             AS hist_prec_{0}_cert_prov 
                  ,t.hist_count_cert_city_num / s.hist_{0}_count_num2             AS hist_prec_{0}_cert_city 
                  ,t.hist_count_card_bin_prov_num / s.hist_{0}_count_num2         AS hist_prec_{0}_card_bin_prov 
                  ,t.hist_count_card_bin_city_num / s.hist_{0}_count_num2         AS hist_prec_{0}_card_bin_city 
                  ,t.hist_count_card_mobile_prov_num / s.hist_{0}_count_num2      AS hist_prec_{0}_card_mobile_prov 
                  ,t.hist_count_card_mobile_city_num / s.hist_{0}_count_num2      AS hist_prec_{0}_card_mobile_city 
                  ,t.hist_count_card_cert_prov_num / s.hist_{0}_count_num2        AS hist_prec_{0}_card_cert_prov 
                  ,t.hist_count_card_cert_city_num / s.hist_{0}_count_num2        AS hist_prec_{0}_card_cert_city 
                  ,t.hist_count_is_one_people_num / s.hist_{0}_count_num2         AS hist_prec_{0}_is_one_people 
                  ,t.hist_count_operation_channel_num / s.hist_{0}_count_num2     AS hist_prec_{0}_operation_channel 
                  ,t.hist_count_pay_scene_num / s.hist_{0}_count_num2             AS hist_prec_{0}_pay_scene 
                  ,t.hist_count_amt_num / s.hist_{0}_count_num2                   AS hist_prec_{0}_amt 
                  ,t.hist_count_is_peer_pay_num / s.hist_{0}_count_num2           AS hist_prec_{0}_is_peer_pay 
              {8} ,t.hist_count_opposing_id_num / s.hist_{0}_count_num2           AS hist_prec_{0}_opposing_id 
                  ,t.hist_count_income_card_no_num / s.hist_{0}_count_num2        AS hist_prec_{0}_income_card_no 
                  ,t.hist_count_income_card_cert_no_num / s.hist_{0}_count_num2   AS hist_prec_{0}_income_card_cert_no 
                  ,t.hist_count_income_card_mobile_num / s.hist_{0}_count_num2    AS hist_prec_{0}_income_card_mobile 
                  ,t.hist_count_income_card_bank_code_num / s.hist_{0}_count_num2 AS hist_prec_{0}_income_card_bank_code 
                  ,t.hist_count_province_num / s.hist_{0}_count_num2              AS hist_prec_{0}_province 
                  ,t.hist_count_city_num / s.hist_{0}_count_num2                  AS hist_prec_{0}_city 
              FROM {1} s 
            LEFT OUTER JOIN  {2} t 
                ON t.event_id = s.event_id
            ;""".format(group_var,df_fenmu,df_fenzi,user_id,client_ip,device_sign,info_1,info_2,opposing_id,user)
         )

# 跑取程序

In [21]:
# Train
train_date = pd.date_range('2017-09-05','2017-11-05').tolist()
# Tset
test_date = pd.date_range('2018-01-05','2018-02-05').tolist()


In [22]:
group_var = 'client_ip'
var = '-'

for i,d in enumerate(train_date+test_date):
    day = datetime.strftime(d,'%Y-%m-%d')
    window_sql(var,day,i,group_var=group_var,date_gap=7,user='kevin_new') 


CREATE TABLE IF NOT EXISTS kevin_new_winFea_client_ip_all_count_7d AS
SELECT v.event_id,s.win_client_ip_to_count_7_num
    FROM(
        SELECT 
            event_id,client_ip
        FROM kevin_new_id_all_win_features    
        WHERE gmt_date=to_date('2017-09-05','yyyy-mm-dd')
    ) v 
    LEFT OUTER JOIN (
        SELECT client_ip, count(*) AS win_client_ip_to_count_7_num
        FROM  kevin_new_id_all_win_features
        WHERE  gmt_date<to_date('2017-09-05','yyyy-mm-dd') and gmt_date>=dateadd(to_date('2017-09-05','yyyy-mm-dd'), -7, 'dd')
                and  client_ip is not NULL
        GROUP BY client_ip
    )s
    ON v.client_ip = s.client_ip;
    
INSERT INTO TABLE kevin_new_winFea_client_ip_all_count_7d 
SELECT v.event_id,s.win_client_ip_to_count_7_num
    FROM(
        SELECT 
            event_id,client_ip
        FROM kevin_new_id_all_win_features    
        WHERE gmt_date=to_date('2017-09-06','yyyy-mm-dd')
    ) v 
    LEFT OUTER JOIN (
        SELECT client_ip, count(*

In [23]:
# distinct count

group_var = 'client_ip'

params = {'user_id':'',
'opposing_id':'',
'client_ip':'--',
'device_sign':'',
'info_1':'',
'info_2':''}

var = '-'

for i,d in enumerate(train_date+test_date):
    day = datetime.strftime(d,'%Y-%m-%d')
    window_distinct_count_sql(var,day,i,group_var=group_var,**params,date_gap=7,user='kevin')



CREATE TABLE IF NOT EXISTS kevin_winFea_client_ip_distinct_count_7d AS
SELECT v.event_id,
         s.win_client_ip_to_user_id_7_num,
        -- s.win_client_ip_to_client_ip_7_num,
            s.win_client_ip_to_network_7_num,
         s.win_client_ip_to_device_sign_7_num,
         s.win_client_ip_to_info_1_7_num,
        s.win_client_ip_to_info_2_7_num,
            s.win_client_ip_to_ip_prov_7_num,
            s.win_client_ip_to_ip_city_7_num,
            s.win_client_ip_to_mobile_oper_platform_7_num,
            s.win_client_ip_to_version_7_num,
            s.win_client_ip_to_card_cert_no_7_num,
            s.win_client_ip_to_cert_prov_7_num,
            s.win_client_ip_to_cert_city_7_num,
            s.win_client_ip_to_card_bin_prov_7_num,
            s.win_client_ip_to_card_bin_city_7_num,
            s.win_client_ip_to_card_mobile_prov_7_num,
            s.win_client_ip_to_card_mobile_city_7_num,
            s.win_client_ip_to_card_cert_prov_7_num,
            s.win_client_ip_to_ca

In [25]:
# 交易金额的合计
var = 'gmt_occur'
# for var in var_ana_list:
#     print("-- ************************{}***************************".format(var))
for i,d in enumerate(train_date+test_date):
    day = datetime.strftime(d,'%Y-%m-%d')
    window_sum_amt_sql(var,day,i,date_gap=7,user='kevin_new') 

CREATE TABLE IF NOT EXISTS kevin_new_winFea_client_ip_sum_amt_7d AS
SELECT v.event_id,
    (case when s.win_sum_amt_7_num is NULL then 0 else s.win_sum_amt_7_num end) as win_sum_amt_7_num
    FROM(
        SELECT 
            event_id,client_ip
        FROM kevin_new_id_all_win_features    
        WHERE gmt_date=to_date('2017-09-05','yyyy-mm-dd')
    ) v 
    LEFT OUTER JOIN (
        SELECT client_ip, sum(amt) AS win_sum_amt_7_num
        FROM  kevin_new_id_all_win_features
        WHERE  gmt_date<to_date('2017-09-05','yyyy-mm-dd') and gmt_date>=dateadd(to_date('2017-09-05','yyyy-mm-dd'), -7, 'dd')
                and  client_ip is not NULL
        GROUP BY client_ip
    )s
    ON v.client_ip = s.client_ip;
    
INSERT INTO TABLE kevin_new_winFea_client_ip_sum_amt_7d 
SELECT v.event_id,
    (case when s.win_sum_amt_7_num is NULL then 0 else s.win_sum_amt_7_num end) as win_sum_amt_7_num
    FROM(
        SELECT 
            event_id,client_ip
        FROM kevin_new_id_all_win_features

### 历史情况

In [26]:
# 分子

# 历史交易统计次数
# group_var = 'opposing_id'
# group_var = 'client_ip'
# group_var = 'device_sign'
# group_var = 'info_1'
group_var = 'info_2'
history_sql(var,var_ana_list,group_var,user='kevin') 


    CREATE TABLE IF NOT EXISTS  kevin_info_2_date_hist_count as        
    SELECT
        s.event_id,s.info_2,

                user_id.user_id, 
                user_id.rn-1  as hist_count_user_id_num,         
            

                client_ip.client_ip, 
                client_ip.rn-1  as hist_count_client_ip_num,         
            

                network.network, 
                network.rn-1  as hist_count_network_num,         
            

                device_sign.device_sign, 
                device_sign.rn-1  as hist_count_device_sign_num,         
            

                info_1.info_1, 
                info_1.rn-1  as hist_count_info_1_num,         
            

                ip_prov.ip_prov, 
                ip_prov.rn-1  as hist_count_ip_prov_num,         
            

                ip_city.ip_city, 
                ip_city.rn-1  as hist_count_ip_city_num,         
            

                mobile_oper_platform.mobile_oper_platform, 
        

In [27]:
# 分母

# group_var = 'opposing_id'
# group_var = 'client_ip'
# group_var = 'device_sign'
# group_var = 'info_1'
group_var = 'info_2'

for group_var in ['opposing_id','client_ip','device_sign','info_1','info_2']:
    history_fenmu_sql(group_var,user='kevin')

CREATE TABLE IF NOT EXISTS kevin_opposing_id_date_gap_count AS
      SELECT t.event_id 
           ,t.opposing_id 
           ,t.gmt_datetime 
           ,t.gmt_datetime_last 
           ,DATEDIFF(gmt_datetime ,gmt_datetime_last ,'hh') AS gmt_time_gap 
           ,(case when (t.rn-1) = 0 then 0.001 else (t.rn-1) end)             AS hist_opposing_id_count_num2 
       FROM (SELECT event_id 
                   ,opposing_id 
                   ,gmt_datetime 
                   ,LAG(gmt_datetime ,1) OVER ( PARTITION BY opposing_id ORDER BY gmt_datetime ) gmt_datetime_last 
                   ,RANK() OVER ( PARTITION BY opposing_id ORDER BY gmt_datetime ) rn                              -- ,
             --     row_number() over(partition by opposing_id order by gmt_datetime ) rn1
               FROM kevin_id_all_win_features 
              WHERE source IN ( 'train' , 'test' ) 
            )t 
    ;
CREATE TABLE IF NOT EXISTS kevin_client_ip_date_gap_count AS
      SELECT t.event_id 
      

In [28]:
# 分子分母相除


# df_fenzi = 'kevin_info_2_date_hist_count' 
# df_fenmu = 'kevin_opposing_id_date_gap_count '
# group_var = 'opposing_id'
# group_var = 'client_ip'
# group_var = 'device_sign'
# group_var = 'info_1'
# group_var = 'info_2'
for group_var in ['client_ip','device_sign','info_1','info_2']:
    params = {  'user_id':'',
                'opposing_id':'',
                'client_ip':'',
                'device_sign':'',
                'info_1':'',
                'info_2':''}
    
    df_fenzi = 'kevin_{}_date_hist_count'.format(group_var) 
    df_fenmu = 'kevin_{}_date_gap_count'.format(group_var)
    params[group_var] = '-- '
    
    history_prec_sql(group_var,df_fenzi,df_fenmu,user='kevin',**params)
    print('-- ','='*60)


            CREATE TABLE IF NOT EXISTS kevin_prec_client_ip_all_data as 
            SELECT s.event_id 
                  ,s.client_ip 
                  ,s.gmt_datetime_last 
                  ,s.hist_client_ip_count_num2
               ,t.hist_count_user_id_num / s.hist_client_ip_count_num2              AS hist_prec_client_ip_user_id
              --  ,t.hist_count_client_ip_num / s.hist_client_ip_count_num2             AS hist_prec_client_ip_client_ip 
                  ,t.hist_count_network_num / s.hist_client_ip_count_num2               AS hist_prec_client_ip_network 
               ,t.hist_count_device_sign_num / s.hist_client_ip_count_num2           AS hist_prec_client_ip_device_sign 
               ,t.hist_count_info_1_num / s.hist_client_ip_count_num2                AS hist_prec_client_ip_info_1 
               ,t.hist_count_info_2_num / s.hist_client_ip_count_num2                AS hist_prec_client_ip_info_2 
                  ,t.hist_count_ip_prov_num / s.hist_client_ip_cou

## 其他特征块

In [16]:
date_gap = 3
for var in var_ana_list:
    print("""
    ,count(DISTINCT {0}) AS win_{0}""".format(var)+'_{3}_num')


    ,count(DISTINCT client_ip) AS win_client_ip_{3}_num

    ,count(DISTINCT network) AS win_network_{3}_num

    ,count(DISTINCT device_sign) AS win_device_sign_{3}_num

    ,count(DISTINCT info_1) AS win_info_1_{3}_num

    ,count(DISTINCT info_2) AS win_info_2_{3}_num

    ,count(DISTINCT ip_prov) AS win_ip_prov_{3}_num

    ,count(DISTINCT ip_city) AS win_ip_city_{3}_num

    ,count(DISTINCT mobile_oper_platform) AS win_mobile_oper_platform_{3}_num

    ,count(DISTINCT version) AS win_version_{3}_num

    ,count(DISTINCT card_cert_no) AS win_card_cert_no_{3}_num

    ,count(DISTINCT cert_prov) AS win_cert_prov_{3}_num

    ,count(DISTINCT cert_city) AS win_cert_city_{3}_num

    ,count(DISTINCT card_bin_prov) AS win_card_bin_prov_{3}_num

    ,count(DISTINCT card_bin_city) AS win_card_bin_city_{3}_num

    ,count(DISTINCT card_mobile_prov) AS win_card_mobile_prov_{3}_num

    ,count(DISTINCT card_mobile_city) AS win_card_mobile_city_{3}_num

    ,count(DISTINCT card_cert_prov) AS 

In [42]:
date_gap = 3
for var in var_ana_list:
    for dg in [1,3,5,7]:
        print("""win_{0}_{1}_num""".format(var,dg))

win_client_ip_1_num
win_client_ip_3_num
win_client_ip_5_num
win_client_ip_7_num
win_network_1_num
win_network_3_num
win_network_5_num
win_network_7_num
win_device_sign_1_num
win_device_sign_3_num
win_device_sign_5_num
win_device_sign_7_num
win_info_1_1_num
win_info_1_3_num
win_info_1_5_num
win_info_1_7_num
win_info_2_1_num
win_info_2_3_num
win_info_2_5_num
win_info_2_7_num
win_ip_prov_1_num
win_ip_prov_3_num
win_ip_prov_5_num
win_ip_prov_7_num
win_ip_city_1_num
win_ip_city_3_num
win_ip_city_5_num
win_ip_city_7_num
win_mobile_oper_platform_1_num
win_mobile_oper_platform_3_num
win_mobile_oper_platform_5_num
win_mobile_oper_platform_7_num
win_version_1_num
win_version_3_num
win_version_5_num
win_version_7_num
win_card_cert_no_1_num
win_card_cert_no_3_num
win_card_cert_no_5_num
win_card_cert_no_7_num
win_cert_prov_1_num
win_cert_prov_3_num
win_cert_prov_5_num
win_cert_prov_7_num
win_cert_city_1_num
win_cert_city_3_num
win_cert_city_5_num
win_cert_city_7_num
win_card_bin_prov_1_num
win_card

In [36]:
### 查看行数特征重要性
print("""
SELECT v.* 
  FROM (SELECT * 
              ,RANK() OVER ( ORDER BY feature_importance11 DESC ) rn1 
              ,RANK() OVER ( ORDER BY feature_importance12 DESC ) rn2 
              ,RANK() OVER ( ORDER BY feature_importance13 DESC ) rn3 
          FROM b_featureimportancelow 
       )v 
 WHERE v.rn1 > 0 
   AND v.rn1 <= 100 
--  WHERE v.rn1 > 100 and  v.rn1 <= 200
--  WHERE v.rn1 > 200 and  v.rn1 <= 300
--  WHERE v.rn1 > 300 and  v.rn1 <= 400
;
""")


SELECT v.* 
  FROM (SELECT * 
              ,RANK() OVER ( ORDER BY feature_importance11 DESC ) rn1 
              ,RANK() OVER ( ORDER BY feature_importance12 DESC ) rn2 
              ,RANK() OVER ( ORDER BY feature_importance13 DESC ) rn3 
          FROM b_featureimportancelow 
       )v 
 WHERE v.rn1 > 0 
   AND v.rn1 <= 100 
--  WHERE v.rn1 > 100 and  v.rn1 <= 200
--  WHERE v.rn1 > 200 and  v.rn1 <= 300
--  WHERE v.rn1 > 300 and  v.rn1 <= 400
;



In [59]:
# var = 'gmt_occur'  # 无用
# group_var = 'opposing_id'
# group_var = 'client_ip'
# group_var = 'device_sign'
# group_var = 'info_1'
# group_var = 'info_2'

# for i,d in enumerate(train_date+test_date):
#     day = datetime.strftime(d,'%Y-%m-%d')
#     window_his_sql(var,day,i,group_var,date_gap=-1,user='kevin')

CREATE TABLE IF NOT EXISTS kevin_histFea_info_2_all_count AS
SELECT v.event_id,s.hist_info_2_count_num
    FROM(
        SELECT 
            event_id,info_2
        FROM kevin_new_id_all_win_features    
        WHERE gmt_date=to_date('2017-09-05','yyyy-mm-dd')
    ) v 
    LEFT OUTER JOIN (
        SELECT info_2, count(*) AS hist_info_2_count_num
        FROM  kevin_new_id_all_win_features
        WHERE  gmt_date<to_date('2017-09-05','yyyy-mm-dd') and  source in ('train','test')
                and  info_2 is not NULL
        GROUP BY info_2
    )s
    ON v.info_2 = s.info_2;
    
INSERT INTO TABLE kevin_histFea_info_2_all_count
SELECT v.event_id,s.hist_info_2_count_num
    FROM(
        SELECT 
            event_id,info_2
        FROM kevin_new_id_all_win_features    
        WHERE gmt_date=to_date('2017-09-06','yyyy-mm-dd')
    ) v 
    LEFT OUTER JOIN (
        SELECT info_2, count(*) AS hist_info_2_count_num
        FROM  kevin_new_id_all_win_features
        WHERE  gmt_date<to_da

In [105]:
# for var in var_ana_list:
#     print("""t.hist_count_{0}_num/s.hist_count_num2 as hist_prec_{0},""".format(var))

t.hist_count_client_ip_num/s.hist_count_num2 as hist_prec_client_ip,
t.hist_count_network_num/s.hist_count_num2 as hist_prec_network,
t.hist_count_device_sign_num/s.hist_count_num2 as hist_prec_device_sign,
t.hist_count_info_1_num/s.hist_count_num2 as hist_prec_info_1,
t.hist_count_info_2_num/s.hist_count_num2 as hist_prec_info_2,
t.hist_count_ip_prov_num/s.hist_count_num2 as hist_prec_ip_prov,
t.hist_count_ip_city_num/s.hist_count_num2 as hist_prec_ip_city,
t.hist_count_mobile_oper_platform_num/s.hist_count_num2 as hist_prec_mobile_oper_platform,
t.hist_count_version_num/s.hist_count_num2 as hist_prec_version,
t.hist_count_card_cert_no_num/s.hist_count_num2 as hist_prec_card_cert_no,
t.hist_count_cert_prov_num/s.hist_count_num2 as hist_prec_cert_prov,
t.hist_count_cert_city_num/s.hist_count_num2 as hist_prec_cert_city,
t.hist_count_card_bin_prov_num/s.hist_count_num2 as hist_prec_card_bin_prov,
t.hist_count_card_bin_city_num/s.hist_count_num2 as hist_prec_card_bin_city,
t.hist_count_c